In [2]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

     |████████████████████████████████| 212.4 MB 51 kB/s 
     |████████████████████████████████| 144 kB 58.8 MB/s 
     |████████████████████████████████| 198 kB 73.0 MB/s 


In [3]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [4]:
from google.colab import files
 
 
uploaded = files.upload()

Saving Suspicious Communication on Social Platforms.csv to Suspicious Communication on Social Platforms.csv


In [5]:
data = pd.read_csv('Suspicious Communication on Social Platforms.csv')

In [6]:
text_list = data.iloc[:, 0].values

In [7]:
spark = sparknlp.start()

In [10]:
MODEL_NAME='classifierdl_use_cyberbullying'

In [11]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_cyberbullying download started this may take some time.
Approximate size to download 21.3 MB
[OK!]


In [12]:
empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result = pipelineModel.transform(df)

In [13]:
result.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment")).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                          |sentiment|
+----------------------------------------------------------------------------------------------------------------------------------+---------+
| Get fucking real dude.                                                                                                           |neutral  |
| She is as dirty as they come  and that crook Rengel  the Dems are so fucking corrupt it's a joke. Make Republicans look like  ...|sexism   |
| why did you fuck it up. I could do it all day too. Let's do it when you have an hour. Ping me later to sched writing a book here.|neutral  |
| Dude they dont finish enclosing the fucking showers. I hate half assed jobs. Whats the reasononing behind it? Makes no sense.    |neutral  |